# Quantization


Quantization is a model optimization technique that reduces the precision of numerical representations, such as the weights and activations in a machine learning model, by using fewer bits. The goal is to improve efficiency (memory usage and computational performance) while maintaining acceptable model accuracy.

For example, in a [`meta-llama/Meta-Llama-3-8B` model](https://huggingface.co/meta-llama/Meta-Llama-3-8B/tree/main), which is an 8-billion model, has 8 billion parameters. Each parameter maps to a number, and each number is represented in a format in a way a classical computer understands. A computer does not understand numbers like humans do, like 1, 100, 3.14 - they use bits. Bits take up memory. More memory means more size. 

[`Meta-Llama-3-8B`](https://huggingface.co/meta-llama/Meta-Llama-3-8B/tree/main) has a total of about 16GB (add the .safetensors files). That means each number is about 2 bytes, or 16 bits. This means a number takes up 16 bits to represent it in a classical computer. The precision defines the number of bytes are used to represent the number. 



## Quantization Impact

### Less Memory Footprint

Quantizing a model reduces the precision of its weights (and often activations) to use fewer bits, which significantly reduces its memory footprint. For example, if [`Meta-Llama-3-8B`](https://huggingface.co/meta-llama/Meta-Llama-3-8B/tree/main) is quantized from 16 bits (`fp16`) to 8 bits (`fp8`), the model size will shrink from 16 GB to 8 GB. This means the quantized model requires half as much memory to load compared to its dense, non-quantized counterpart. This is especially beneficial for deploying LLMs on memory-constrained devices

> Any Quantization -> Less memory

### Faster Loading From Disk

Quantized models require fewer bits, which reduces the amount of data that must be transferred between storage and memory (e.g., from disk to CPU/GPU memory or through caches such as L2). This reduced data size improves loading times.
This is particularly important for LLMs during inference, as weights from different layers are often loaded and unloaded repeatedly. When the time required to transfer data is greater than the time to compute on that data, the system becomes memory-bound. Quantization helps mitigate this bottleneck by reducing data transfer time. 

> Any Quantization -> Less transfer time


## Downsides

Quantization is an optimization technique that comes with trade-offs. In any machine learning or LLM model, three key components must be considered when evaluating the impact of quantization:

**Accuracy**: How well the model performs on the task, often measured by metrics like precision, recall, or perplexity. Quantization can reduce accuracy because lowering the precision of weights and activations may introduce rounding errors and reduce the model’s ability to represent fine-grained patterns. Ex. weight in `fp16` -> 3.14, weight in `int4` -> 3.


**Inference Speed**: How quickly the model generates output. Quantization reduces the size of weights and activations, enabling faster data transfers and computation on supported hardware, which improves inference speed.

**Size**: The memory footprint of the model. Quantization significantly reduces the memory required to store and load the model.

## Trade-offs
Optimizing one of these components often impacts the others, requiring careful trade-offs to maintain overall performance:

**Improving Accuracy**: Increasing the number of parameters or retaining higher precision can boost accuracy but comes at the cost of larger model size and slower inference.

**Reducing Size**: Lowering the precision of weights can shrink its size and improve inference speed but risks losing accuracy, as the model may struggle to capture complex patterns.

**Increasing Inference Speed**: Quantization reduces the data to be processed, improving speed, but may compromise accuracy due to reduced precision.

## Mitigating Downsides

While quantization provides significant memory and speed benefits, it may also introduce a small loss in model accuracy due to the reduced precision. Post-Training Quantization (PTQ) is one technique used to minimize the downside.



# PTQ

Post training quantization (PTQ) is the process of reducing the precision of model weights for an already-trained model, such as pretrained model. 


In [ ]:
# Minimizing Quantization Error

# TODO

## PTQ Impact

### Faster Inference on Supported Machines

Quantization can significantly improve inference speed on hardware that supports lower-precision computations. Many modern GPUs and accelerators, such as NVIDIA's H100 GPUs, support specialized formats like `fp8`. When these formats are natively supported, the hardware can process computations directly, resulting in faster inference.

However, on hardware that does not support the target format, the quantized representation must first be converted to a supported precision (usually higher precision, `fp16`, `fp32` etc.) before computations can occur. This conversion introduces overhead, which can negate the performance gains of quantization. Therefore, quantization is most effective when used with hardware designed for lower-precision arithmetic.

> PTQ + supported hardware -> faster inference


### Faster Inference Using Specialized Kernels

To leverage the hardware computation for LLM workloads - matrix multiplications - specialized kernels can be integrated for a specific precision. Kernels are computer instructions that provide details on data manipulation and computation (ex. apply weight shuffling to align weight entries for memory access patterns for NVIDIA tensor cores to maximize throughput and minimize loading latency). One example is [Machete kernel](https://neuralmagic.com/blog/introducing-machete-a-mixed-input-gemm-kernel-optimized-for-nvidia-hopper-gpus/?utm_source=chatgpt.com), specifically optimized for H100 GPUs and model weights quantized to 4 bits. Another is Marlin, which contain instructions for A100 GPUs for `int4` weights and `fp16` activations.


To maximize the computational potential of modern hardware for LLM workloads — primarily matrix multiplications — specialized kernels are employed. Kernels are sets of computer instructions designed to optimize how data is manipulated and computations are performed. Ex. techniques like weight shuffling reorganizes weight data to align with memory access patterns, allowing NVIDIA Tensor Cores to maximize throughput and minimize loading latency. Specialized Kernels include:

* **Machete**:
Optimized for NVIDIA H100 GPUs. Designed for models with weights quantized to `int4`. By leveraging hardware features like advanced Tensor Core instructions (wgmma) and efficient memory access strategies, Machete dramatically improves inference speed and reduces memory usage - mediam time to first token to <250ms and median time per output token to <100ms with 1xH100 on Llama 3.1-70B.

* **Marlin**:
Tailored for NVIDIA A100 GPUs, Marlin supports mixed-precision computations with INT4 weights and FP16 activations. It provides optimized instructions to handle these lower-precision formats efficiently, enabling faster inference on hardware that supports these precision formats.

> PTQ + supported hardware + specific kernel -> much faster inference

Using quantization techniques, along with precison supported hardware and specialized kernels, it demonstrates how hardware and software innovations work together to handle the growing computational demands of LLMs, achieving faster inference times while using less memory.`m

## Precision Formats

TODO

When a number is stored in 16 bit format, it can either be in 
1. fp16
2. int16
3. bf16
4. uint16



In [ ]:
## Algorithms
TODO

* W4a16
w8a16
w8a8

gptq
smoothquant..


In [ ]:
# When to use which algorithms

# Which algo is supported on which hardware


# Which algo is supported on whihc kernel
# How do the kernels work
# # What are the impact of using the kernel
# In what situations do you use what
- in low QPS, use weight only - not much speed up in high QPS
- in high QPS or offline, use activation quantization
      A100 - w8a8 chan weights, dynamic per token act
      H100 - w8a8 fp8 chan weight, dpt